In [ ]:
# HTML generator with client-side include fragments
# This cell writes two fragment files (includes/header.html, includes/footer.html)
# and generates ParentList_Marksheet.html which uses a small JS to include them.

import pandas as pd
import requests
import io
import os
from datetime import datetime

# Configuration
sheet_id = "1ZHYLwglcO_uuQeZsfEKieOyq4tJzTKqlAQA99ZcaHTw"
gid = "481425432"
output_html = "ParentList_Marksheet.html"
includes_dir = os.path.join(os.path.dirname(output_html) or '.', 'includes')
os.makedirs(includes_dir, exist_ok=True)

export_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"
print('Downloading CSV from:', export_url)

resp = requests.get(export_url, timeout=15)
resp.raise_for_status()
text = resp.content.decode('utf-8', errors='replace')
df = pd.read_csv(io.StringIO(text))

# Normalize column names
cols = {c.lower().strip(): c for c in df.columns}

def find_col(possible):
    for p in possible:
        if p.lower() in cols:
            return cols[p.lower()]
    return None

parent_col = find_col(["parent name", "parent", "parent_name"])
student_col = find_col(["student name", "student", "student_name"])
time_col = find_col(["timestamp", "submitted on", "submittedon", "submitted_on"])

if parent_col is None:
    df['Parent Name'] = ''
    parent_col = 'Parent Name'
if student_col is None:
    df['Student Name'] = ''
    student_col = 'Student Name'
if time_col is None:
    df['Submitted On'] = ''
    time_col = 'Submitted On'

# Keep canonical columns
df = df[[parent_col, student_col, time_col]].copy()
df.columns = ['Parent Name', 'Student Name', 'Submitted On']

# Drop empty rows
df = df.dropna(how='all')
mask = (df['Parent Name'].astype(str).str.strip() != '') | (df['Student Name'].astype(str).str.strip() != '')
df = df[mask].reset_index(drop=True)

# Sort and add sequence
df['Parent_sort'] = df['Parent Name'].astype(str).str.lower()
df = df.sort_values(['Parent_sort', 'Student Name'], ascending=[True, True]).reset_index(drop=True)
df = df.drop(columns=['Parent_sort'])
df.insert(0, 'No', range(1, len(df) + 1))

# Helper to escape HTML
def esc(s):
    return (str(s).replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;'))

rows = []
for _, r in df.iterrows():
    rows.append(f"<tr><td class=\"sequence-number\">{int(r['No'])}</td><td>{esc(r['Parent Name'])}</td><td>{esc(r['Student Name'])}</td><td>{esc(r['Submitted On'])}</td></tr>")

# Write include fragments
header_frag = '''
<div class="page-header">
  <h1>Parent and Student List</h1>
  <div style="text-align:center; margin-bottom:12px">
    <input id="searchInput" type="text" placeholder="Search parent, student or timestamp..." onkeyup="searchTable()" />
  </div>
</div>
'''

footer_frag = f'''
<div class="footer">Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</div>
'''

with open(os.path.join(includes_dir, 'header.html'), 'w', encoding='utf-8') as f:
    f.write(header_frag)
with open(os.path.join(includes_dir, 'footer.html'), 'w', encoding='utf-8') as f:
    f.write(footer_frag)

# Main HTML (references the fragments via data-include)
css = '''
body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; background:#f5f5f5; padding:20px; }
.container{ max-width:1200px; margin:0 auto; background:#fff; padding:20px; border-radius:10px; box-shadow:0 0 20px rgba(0,0,0,0.08); }
h1{ text-align:center; color:#2c3e50; margin:0 0 12px }
#searchInput{ width:80%; max-width:420px; padding:10px 14px; border-radius:25px; border:2px solid #3498db; }
table{ width:100%; border-collapse:collapse; margin-top:16px }
th,td{ padding:12px 10px; border-bottom:1px solid #eee; text-align:left }
th{ background:#3498db; color:#fff; text-transform:uppercase }
tr:nth-child(even){ background:#fafafa }
.sequence-number{ color:#7f8c8d; width:50px }
.footer{ text-align:center; color:#7f8c8d; margin-top:14px }
.no-results{ text-align:center; color:#7f8c8d; padding:12px; display:none }
@media (max-width:600px){ table{ display:block; overflow-x:auto } }
'''

main_html = f'''<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>ParentList Marksheet</title>
<style>{css}</style>
</head>
<body>
  <div class="container">
    <div data-include="includes/header.html"></div>
    <table id="dataTable">
      <thead>
        <tr><th class="sequence-number">#</th><th>Parent Name</th><th>Student Name</th><th>Submitted On</th></tr>
      </thead>
      <tbody>
{chr(10).join(rows)}
      </tbody>
    </table>
    <div id="noResults" class="no-results">No matching results found</div>
    <div data-include="includes/footer.html"></div>
  </div>

<script>
// Simple client-side include: loads elements with data-include attribute
async function includeFragments() 
{
  const els = document.querySelectorAll('[data-include]');
  for (const el of els) {
    const url = el.getAttribute('data-include');
    try {
      const r = await fetch(url);
      if (r.ok) el.innerHTML = await r.text();
      else el.innerHTML = '';
    } catch (e) {
      el.innerHTML = '';
    }
  }
}

// Search function (works after header is included)
function searchTable() 
{
  const input = document.getElementById('searchInput');
  const filter = (input && input.value ? input.value.toLowerCase() : '');
  const rows = document.getElementById('dataTable').getElementsByTagName('tr');
  let found = false;
  for (let i=1; i<rows.length; i++){
    const cols = rows[i].getElementsByTagName('td');
    if (!cols || cols.length < 4) continue;
    const p = (cols[1].textContent || cols[1].innerText).toLowerCase();
    const s = (cols[2].textContent || cols[2].innerText).toLowerCase();
    const t = (cols[3].textContent || cols[3].innerText).toLowerCase();
    if (p.indexOf(filter) > -1 || s.indexOf(filter) > -1 || t.indexOf(filter) > -1) {
      rows[i].style.display = '';
      found = true;
    } else {
      rows[i].style.display = 'none';
    }
  }
  document.getElementById('noResults').style.display = found ? 'none' : 'block';
}

// Run includes on load
includeFragments();
</script>
</body>
</html>
'''

with open(output_html, 'w', encoding='utf-8') as f:
    f.write(main_html)

print(f"Wrote {output_html} and include fragments in {includes_dir} ({len(df)} rows). Note: when opening the HTML locally some browsers block fetching local include files; if that happens serve the folder with a simple HTTP server: `python -m http.server 8000` and open http://localhost:8000/{output_html}")


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2460331695.py, line 133)